# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_val = pd.read_csv('data/y_val.csv')
y_test = pd.read_csv('data/y_test.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

In [5]:
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

2022-03-14 20:50:06.067657: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-14 20:50:06.068028: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro
Epoch 1/150


2022-03-14 20:50:06.686133: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-14 20:50:06.884667: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 2s 12ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
12/33 [=========>....................] - ETA: 0s - loss: nan - mse: nan

2022-03-14 20:50:08.324895: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 10/150
33/33 [==============================] - 0s 5ms

33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 78/150
33/33 [==============================] -

33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 146/150
33/33 [=========================

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [7]:
# Numeric column names
numeric_columns = X_train_numeric.columns

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [8]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 

In [10]:
# Train the model 
normalized_input_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
26/33 [======================>.......] - ETA: 0s - loss: nan - mse: nan                         

2022-03-14 20:54:26.727539: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 8ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
24/33 [====================>.........] - ETA: 0s - loss: nan - mse: nan

2022-03-14 20:54:26.981580: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 10/150
33/33 [==============================] - 0s 5ms

33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 78/150
33/33 [==============================] -

33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [==============================] - 0s 5ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 146/150
33/33 [=========================

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [11]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [12]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, 
                   y_train_scaled, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val_scaled))

Epoch 1/150
26/33 [======================>.......] - ETA: 0s - loss: 0.4572 - mse: 0.4572

2022-03-14 20:56:00.182142: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 10ms/step - loss: 0.4091 - mse: 0.4091 - val_loss: 0.1493 - val_mse: 0.1493
Epoch 2/150
26/33 [======================>.......] - ETA: 0s - loss: 0.2231 - mse: 0.2231

2022-03-14 20:56:00.510791: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 5ms/step - loss: 0.2185 - mse: 0.2185 - val_loss: 0.1314 - val_mse: 0.1314
Epoch 3/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1868 - mse: 0.1868 - val_loss: 0.1224 - val_mse: 0.1224
Epoch 4/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1637 - mse: 0.1637 - val_loss: 0.1169 - val_mse: 0.1169
Epoch 5/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1465 - mse: 0.1465 - val_loss: 0.1228 - val_mse: 0.1228
Epoch 6/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1319 - mse: 0.1319 - val_loss: 0.1158 - val_mse: 0.1158
Epoch 7/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1262 - mse: 0.1262 - val_loss: 0.1151 - val_mse: 0.1151
Epoch 8/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1160 - mse: 0.1160 - val_loss: 0.1106 - val_mse: 0.1106
Epoch 9/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1082 - mse: 0.1082 -

33/33 [==============================] - 0s 5ms/step - loss: 0.0223 - mse: 0.0223 - val_loss: 0.1152 - val_mse: 0.1152
Epoch 65/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0218 - mse: 0.0218 - val_loss: 0.1154 - val_mse: 0.1154
Epoch 66/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0213 - mse: 0.0213 - val_loss: 0.1164 - val_mse: 0.1164
Epoch 67/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0214 - mse: 0.0214 - val_loss: 0.1166 - val_mse: 0.1166
Epoch 68/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0211 - mse: 0.0211 - val_loss: 0.1148 - val_mse: 0.1148
Epoch 69/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0205 - mse: 0.0205 - val_loss: 0.1158 - val_mse: 0.1158
Epoch 70/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0201 - mse: 0.0201 - val_loss: 0.1173 - val_mse: 0.1173
Epoch 71/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0202 - mse: 0

Epoch 126/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0105 - mse: 0.0105 - val_loss: 0.1239 - val_mse: 0.1239
Epoch 127/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0102 - mse: 0.0102 - val_loss: 0.1209 - val_mse: 0.1209
Epoch 128/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0102 - mse: 0.0102 - val_loss: 0.1224 - val_mse: 0.1224
Epoch 129/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0101 - mse: 0.0101 - val_loss: 0.1217 - val_mse: 0.1217
Epoch 130/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0099 - mse: 0.0099 - val_loss: 0.1229 - val_mse: 0.1229
Epoch 131/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0099 - mse: 0.0099 - val_loss: 0.1225 - val_mse: 0.1225
Epoch 132/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0098 - mse: 0.0098 - val_loss: 0.1227 - val_mse: 0.1227
Epoch 133/150
33/33 [==============================] - 0s 5ms/step - 

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [13]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 3ms/step - loss: 0.0078 - mse: 0.0078


[0.007799358572810888, 0.007799358572810888]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [14]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 4ms/step - loss: 0.1248 - mse: 0.1248


[0.12475598603487015, 0.12475598603487015]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [15]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

2022-03-14 20:58:25.613478: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


27756.341640438062

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'`  

In [16]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,), kernel_initializer='he_normal'))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Epoch 1/150
26/33 [======================>.......] - ETA: 0s - loss: 0.5017 - mse: 0.5017

2022-03-14 21:00:02.336250: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 8ms/step - loss: 0.4565 - mse: 0.4565 - val_loss: 0.2199 - val_mse: 0.2199
Epoch 2/150
25/33 [=====================>........] - ETA: 0s - loss: 0.2957 - mse: 0.2957

2022-03-14 21:00:02.580912: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 5ms/step - loss: 0.2599 - mse: 0.2599 - val_loss: 0.1766 - val_mse: 0.1766
Epoch 3/150
33/33 [==============================] - 0s 5ms/step - loss: 0.2172 - mse: 0.2172 - val_loss: 0.1703 - val_mse: 0.1703
Epoch 4/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1817 - mse: 0.1817 - val_loss: 0.1585 - val_mse: 0.1585
Epoch 5/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1644 - mse: 0.1644 - val_loss: 0.1511 - val_mse: 0.1511
Epoch 6/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1525 - mse: 0.1525 - val_loss: 0.1506 - val_mse: 0.1506
Epoch 7/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1389 - mse: 0.1389 - val_loss: 0.1527 - val_mse: 0.1527
Epoch 8/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1296 - mse: 0.1296 - val_loss: 0.1474 - val_mse: 0.1474
Epoch 9/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1234 - mse: 0.1234 -

33/33 [==============================] - 0s 5ms/step - loss: 0.0247 - mse: 0.0247 - val_loss: 0.1545 - val_mse: 0.1545
Epoch 65/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0240 - mse: 0.0240 - val_loss: 0.1547 - val_mse: 0.1547
Epoch 66/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0240 - mse: 0.0240 - val_loss: 0.1554 - val_mse: 0.1554
Epoch 67/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0232 - mse: 0.0232 - val_loss: 0.1547 - val_mse: 0.1547
Epoch 68/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0228 - mse: 0.0228 - val_loss: 0.1542 - val_mse: 0.1542
Epoch 69/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0229 - mse: 0.0229 - val_loss: 0.1548 - val_mse: 0.1548
Epoch 70/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0222 - mse: 0.0222 - val_loss: 0.1568 - val_mse: 0.1568
Epoch 71/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0218 - mse: 0

Epoch 126/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0102 - mse: 0.0102 - val_loss: 0.1665 - val_mse: 0.1665
Epoch 127/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0101 - mse: 0.0101 - val_loss: 0.1665 - val_mse: 0.1665
Epoch 128/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0099 - mse: 0.0099 - val_loss: 0.1671 - val_mse: 0.1671
Epoch 129/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0098 - mse: 0.0098 - val_loss: 0.1682 - val_mse: 0.1682
Epoch 130/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0097 - mse: 0.0097 - val_loss: 0.1673 - val_mse: 0.1673
Epoch 131/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0097 - mse: 0.0097 - val_loss: 0.1678 - val_mse: 0.1678
Epoch 132/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0095 - mse: 0.0095 - val_loss: 0.1674 - val_mse: 0.1674
Epoch 133/150
33/33 [==============================] - 0s 5ms/step - 

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [17]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 3ms/step - loss: 0.0075 - mse: 0.0075


[0.007514104247093201, 0.007514104247093201]

Evaluate the model (`he_model`) on validate data (`X_train` and `y_train_scaled`) 

In [18]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 4ms/step - loss: 0.1690 - mse: 0.1690


[0.1689903885126114, 0.1689903885126114]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 

In [19]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,), kernel_initializer='lecun_normal'))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
24/33 [====================>.........] - ETA: 0s - loss: 0.5683 - mse: 0.5683

2022-03-14 21:01:34.132147: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 8ms/step - loss: 0.4838 - mse: 0.4838 - val_loss: 0.1975 - val_mse: 0.1975
Epoch 2/150
24/33 [====================>.........] - ETA: 0s - loss: 0.2490 - mse: 0.2490

2022-03-14 21:01:34.385988: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 5ms/step - loss: 0.2420 - mse: 0.2420 - val_loss: 0.1540 - val_mse: 0.1540
Epoch 3/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1930 - mse: 0.1930 - val_loss: 0.1403 - val_mse: 0.1403
Epoch 4/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1662 - mse: 0.1662 - val_loss: 0.1367 - val_mse: 0.1367
Epoch 5/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1501 - mse: 0.1501 - val_loss: 0.1341 - val_mse: 0.1341
Epoch 6/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1329 - mse: 0.1329 - val_loss: 0.1214 - val_mse: 0.1214
Epoch 7/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1282 - mse: 0.1282 - val_loss: 0.1262 - val_mse: 0.1262
Epoch 8/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1139 - mse: 0.1139 - val_loss: 0.1268 - val_mse: 0.1268
Epoch 9/150
33/33 [==============================] - 0s 5ms/step - loss: 0.1077 - mse: 0.1077 -

33/33 [==============================] - 0s 5ms/step - loss: 0.0205 - mse: 0.0205 - val_loss: 0.1475 - val_mse: 0.1475
Epoch 65/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0204 - mse: 0.0204 - val_loss: 0.1519 - val_mse: 0.1519
Epoch 66/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0208 - mse: 0.0208 - val_loss: 0.1495 - val_mse: 0.1495
Epoch 67/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0197 - mse: 0.0197 - val_loss: 0.1510 - val_mse: 0.1510
Epoch 68/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0198 - mse: 0.0198 - val_loss: 0.1500 - val_mse: 0.1500
Epoch 69/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0200 - mse: 0.0200 - val_loss: 0.1519 - val_mse: 0.1519
Epoch 70/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0190 - mse: 0.0190 - val_loss: 0.1502 - val_mse: 0.1502
Epoch 71/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0188 - mse: 0

Epoch 126/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0094 - mse: 0.0094 - val_loss: 0.1648 - val_mse: 0.1648
Epoch 127/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0092 - mse: 0.0092 - val_loss: 0.1617 - val_mse: 0.1617
Epoch 128/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0089 - mse: 0.0089 - val_loss: 0.1601 - val_mse: 0.1601
Epoch 129/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0091 - mse: 0.0091 - val_loss: 0.1597 - val_mse: 0.1597
Epoch 130/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0089 - mse: 0.0089 - val_loss: 0.1606 - val_mse: 0.1606
Epoch 131/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0089 - mse: 0.0089 - val_loss: 0.1618 - val_mse: 0.1618
Epoch 132/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0088 - mse: 0.0088 - val_loss: 0.1633 - val_mse: 0.1633
Epoch 133/150
33/33 [==============================] - 0s 5ms/step - 

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [20]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 3ms/step - loss: 0.0069 - mse: 0.0069


[0.006881191395223141, 0.006881191395223141]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [22]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 4ms/step - loss: 0.1615 - mse: 0.1615


[0.16152553260326385, 0.16152551770210266]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

In [23]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer='rmsprop', loss='mse', metrics=['mse'])

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150


2022-03-14 21:03:14.288627: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 1s 8ms/step - loss: 0.2881 - mse: 0.2881 - val_loss: 0.1198 - val_mse: 0.1198
Epoch 2/150
21/33 [==================>...........] - ETA: 0s - loss: 0.2020 - mse: 0.2020

2022-03-14 21:03:14.815973: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 6ms/step - loss: 0.1575 - mse: 0.1575 - val_loss: 0.0820 - val_mse: 0.0820
Epoch 3/150
33/33 [==============================] - 0s 6ms/step - loss: 0.1128 - mse: 0.1128 - val_loss: 0.0942 - val_mse: 0.0942
Epoch 4/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0937 - mse: 0.0937 - val_loss: 0.1017 - val_mse: 0.1017
Epoch 5/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0805 - mse: 0.0805 - val_loss: 0.1219 - val_mse: 0.1219
Epoch 6/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0642 - mse: 0.0642 - val_loss: 0.1000 - val_mse: 0.1000
Epoch 7/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0556 - mse: 0.0556 - val_loss: 0.1253 - val_mse: 0.1253
Epoch 8/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0483 - mse: 0.0483 - val_loss: 0.1124 - val_mse: 0.1124
Epoch 9/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0436 - mse: 0.0436 -

33/33 [==============================] - 0s 6ms/step - loss: 0.0076 - mse: 0.0076 - val_loss: 0.1205 - val_mse: 0.1205
Epoch 65/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0076 - mse: 0.0076 - val_loss: 0.1191 - val_mse: 0.1191
Epoch 66/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0113 - mse: 0.0113 - val_loss: 0.1305 - val_mse: 0.1305
Epoch 67/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0074 - mse: 0.0074 - val_loss: 0.1160 - val_mse: 0.1160
Epoch 68/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0099 - mse: 0.0099 - val_loss: 0.1220 - val_mse: 0.1220
Epoch 69/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0047 - mse: 0.0047 - val_loss: 0.1197 - val_mse: 0.1197
Epoch 70/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0084 - mse: 0.0084 - val_loss: 0.1160 - val_mse: 0.1160
Epoch 71/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0084 - mse: 0

Epoch 126/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0065 - mse: 0.0065 - val_loss: 0.1167 - val_mse: 0.1167
Epoch 127/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0055 - mse: 0.0055 - val_loss: 0.1299 - val_mse: 0.1299
Epoch 128/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0039 - mse: 0.0039 - val_loss: 0.1114 - val_mse: 0.1114
Epoch 129/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0063 - mse: 0.0063 - val_loss: 0.1112 - val_mse: 0.1112
Epoch 130/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0068 - mse: 0.0068 - val_loss: 0.1114 - val_mse: 0.1114
Epoch 131/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0056 - mse: 0.0056 - val_loss: 0.1071 - val_mse: 0.1071
Epoch 132/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0037 - mse: 0.0037 - val_loss: 0.1102 - val_mse: 0.1102
Epoch 133/150
33/33 [==============================] - 0s 6ms/step - 

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [24]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 3ms/step - loss: 0.0025 - mse: 0.0025


[0.0025379504077136517, 0.0025379504077136517]

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [25]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 4ms/step - loss: 0.1098 - mse: 0.1098


[0.10980573296546936, 0.10980573296546936]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric  

In [26]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model

adam_model.compile(optimizer='Adam', metrics=['mse'], loss= 'mse')

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150


2022-03-14 21:04:19.124363: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 1s 8ms/step - loss: 0.3091 - mse: 0.3091 - val_loss: 0.1301 - val_mse: 0.1301
Epoch 2/150
21/33 [==================>...........] - ETA: 0s - loss: 0.1662 - mse: 0.1662

2022-03-14 21:04:19.694615: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


33/33 [==============================] - 0s 6ms/step - loss: 0.1571 - mse: 0.1571 - val_loss: 0.1029 - val_mse: 0.1029
Epoch 3/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0992 - mse: 0.0992 - val_loss: 0.1075 - val_mse: 0.1075
Epoch 4/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0715 - mse: 0.0715 - val_loss: 0.1051 - val_mse: 0.1051
Epoch 5/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0530 - mse: 0.0530 - val_loss: 0.1119 - val_mse: 0.1119
Epoch 6/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0416 - mse: 0.0416 - val_loss: 0.1283 - val_mse: 0.1283
Epoch 7/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0363 - mse: 0.0363 - val_loss: 0.1197 - val_mse: 0.1197
Epoch 8/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0287 - mse: 0.0287 - val_loss: 0.1346 - val_mse: 0.1346
Epoch 9/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0266 - mse: 0.0266 -

Epoch 64/150
33/33 [==============================] - 0s 6ms/step - loss: 7.8186e-04 - mse: 7.8186e-04 - val_loss: 0.1226 - val_mse: 0.1226
Epoch 65/150
33/33 [==============================] - 0s 6ms/step - loss: 9.2132e-04 - mse: 9.2132e-04 - val_loss: 0.1209 - val_mse: 0.1209
Epoch 66/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0018 - mse: 0.0018 - val_loss: 0.1320 - val_mse: 0.1320
Epoch 67/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0026 - mse: 0.0026 - val_loss: 0.1091 - val_mse: 0.1091
Epoch 68/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0055 - mse: 0.0055 - val_loss: 0.1260 - val_mse: 0.1260
Epoch 69/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0054 - mse: 0.0054 - val_loss: 0.1255 - val_mse: 0.1255
Epoch 70/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0032 - mse: 0.0032 - val_loss: 0.1210 - val_mse: 0.1210
Epoch 71/150
33/33 [==============================] - 0s 6ms

33/33 [==============================] - 0s 6ms/step - loss: 0.0050 - mse: 0.0050 - val_loss: 0.1148 - val_mse: 0.1148
Epoch 125/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0042 - mse: 0.0042 - val_loss: 0.1260 - val_mse: 0.1260
Epoch 126/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0027 - mse: 0.0027 - val_loss: 0.1120 - val_mse: 0.1120
Epoch 127/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0020 - mse: 0.0020 - val_loss: 0.1305 - val_mse: 0.1305
Epoch 128/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0020 - mse: 0.0020 - val_loss: 0.1135 - val_mse: 0.1135
Epoch 129/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0025 - mse: 0.0025 - val_loss: 0.1321 - val_mse: 0.1321
Epoch 130/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0027 - mse: 0.0027 - val_loss: 0.1072 - val_mse: 0.1072
Epoch 131/150
33/33 [==============================] - 0s 6ms/step - loss: 0.0026 -

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [27]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 3ms/step - loss: 0.0018 - mse: 0.0018


[0.001817541429772973, 0.001817541429772973]

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [28]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 4ms/step - loss: 0.1141 - mse: 0.1141


[0.11413362622261047, 0.11413362622261047]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [29]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

5/5 [==============================] - 0s 6ms/step - loss: 0.1561 - mse: 0.1561


[0.1561087965965271, 0.1561087965965271]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [33]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

31048.825710163022

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.